# Hướng Dẫn Chạy Dự Án Trên Cloud
Notebook này hướng dẫn bạn chạy mã nguồn và notebook trên các nền tảng cloud phổ biến.


## 1) Tùy Chọn Nền Tảng Cloud
Các nền tảng phổ biến để chạy Jupyter notebooks và mã Python:
- Google Colab: Miễn phí, tích hợp GPU/TPU tùy đợt, thuận tiện upload và chia sẻ.
- Kaggle Notebooks (Kernels): Miễn phí, có GPU theo quota, dễ gắn dataset Kaggle.
- AWS SageMaker: Trả phí, mạnh mẽ cho sản xuất, notebook instance cấu hình được.
- Azure (Azure ML/Notebooks): Trả phí hoặc hạn mức, tích hợp hệ sinh thái Azure.

Chọn Colab nếu bạn muốn nhanh, miễn phí và dễ dùng.

## 2) Thiết Lập Google Colab
- Truy cập: https://colab.google/
- Đăng nhập tài khoản Google, chọn "New Notebook".
- Đổi runtime nếu cần: Runtime → Change runtime type → chọn GPU (nếu cần).
- Colab dùng Python 3.x; để chạy TensorFlow 1.x, dùng lệnh `%tensorflow_version 1.x`.

## 3) Upload & Chạy Code Trên Google Colab
Bạn có 2 cách để đưa dữ liệu vào Colab:
- Upload trực tiếp qua UI: biểu tượng thư mục (Files) → Upload.
- Mount Google Drive: giúp dùng dữ liệu từ Drive.

Chạy các ô sau trong Colab để thiết lập môi trường và repo:

## 3a) Dọn dẹp môi trường Colab (tuỳ chọn nhưng khuyến nghị)
Colab cài sẵn nhiều thư viện có thể xung đột. Chạy ô sau để gỡ các gói không cần thiết cho dự án này trước khi cài đặt lại phiên bản phù hợp.

In [ ]:
# Cài TensorFlow 2.19.x và thư viện tương thích (tránh xung đột Colab)
!pip install -q --upgrade --force-reinstall \
  tensorflow==2.19.0 tf-keras==2.19.0 tensorflow-text==2.19.0 \
  tf_slim==1.1.0 numpy==1.26.4 opencv-python-headless==4.7.0.72 \
  scikit-image scipy matplotlib tqdm requests==2.32.4 pillow==11.0.0 rich==13.7.1

import tensorflow as tf, numpy as np, cv2, skimage, scipy, tqdm, matplotlib
print('TF:', tf.__version__, 'NP:', np.__version__)

# Chuyển sang chế độ tương thích TF1 (graph mode)
tf.compat.v1.disable_eager_execution()
print('Eager execution:', tf.executing_eagerly())

In [ ]:
# Clone repo và chuyển thư mục
!rm -rf ComparisonDetector
!git clone https://github.com/CVIU-CSU/ComparisonDetector
%cd /content/ComparisonDetector
!ls -la

In [ ]:
# Tạo thư mục dữ liệu và hướng dẫn mount/upload
from google.colab import drive
drive.mount('/content/drive')
!mkdir -p /content/data/tct
print('Hãy đặt TFRecord vào /content/data/tct như train.tfrecord, test.tfrecord và labels.tsv')

In [ ]:
# Tạo/cập nhật configs/config.py cho Colab
import os, textwrap
os.makedirs('configs', exist_ok=True)
cfg = textwrap.dedent('''\
DATA_DIR = '/content/data/tct'
CHECKPOINT_DIR = ''  # để trống để train từ đầu
''')
open('configs/config.py', 'w').write(cfg)
print(open('configs/config.py').read())

In [ ]:
# (Tùy chọn) Vá nhanh reference.py nếu dùng scipy.misc.imresize
import os, re
p = 'reference.py'
if os.path.exists(p):
    s = open(p,'r').read()
    # Chuẩn hoá: dùng skimage.transform.resize; giữ giá trị pixel
    s = re.sub(r"import scipy.*", "import skimage.transform", s)
    s = s.replace('scipy.misc.imresize', 'skimage.transform.resize')
    # Nếu cần, thêm preserve_range và anti_aliasing khi resize
    if 'skimage.transform.resize' in s and 'preserve_range' not in s:
        s = s.replace('skimage.transform.resize(', 'skimage.transform.resize(')
    open(p,'w').write(s)
    print('Đã vá reference.py dùng skimage.transform.resize')
else:
    print('Không tìm thấy reference.py; bỏ qua')

In [ ]:
# Chạy training (cần có /content/data/tct/train.tfrecord, test.tfrecord, labels.tsv)
# Lưu ý: với TF2 + compat.v1, phần lớn API TF1 sẽ hoạt động nếu không dùng tf.contrib ngoài slim
!python tools/train.py

## 4) Thiết Lập Azure Notebooks (Azure ML)
- Tạo tài khoản Azure, vào Azure ML Studio.
- Tạo compute instance (máy ảo) cho notebook, chọn kích thước theo nhu cầu GPU/CPU.
- Mở Jupyter/VS Code trong Azure ML, upload repo và dữ liệu.
- Cài thư viện bằng `pip` trong terminal của instance, rồi chạy notebook/shell tương tự Colab.

## 5) Thiết Lập AWS SageMaker
- Vào AWS Console → SageMaker → Notebook instances.
- Tạo instance với role có quyền truy cập S3.
- Kết nối JupyterLab, clone repo từ GitHub hoặc tải lên.
- Lưu dữ liệu vào S3 rồi mount/đọc từ S3; cài thư viện bằng `pip`.
- Chạy notebook/scripts tương tự Colab.

## 6) Cài Thư Viện Bằng Pip (Mẫu Chung)
Dù ở nền tảng nào, bạn thường cài thư viện bằng pip trong notebook hoặc terminal:
```python
!pip install tensorflow==1.15.0 tf_slim==1.1.0 numpy==1.19.5 opencv-python-headless scikit-image scipy matplotlib tqdm
```
Ghi chú: Dự án này dựa trên TensorFlow 1.x nên cần đúng phiên bản.

## 7) Lưu & Chia Sẻ Notebook
- Colab: File → Save a copy in Drive để lưu; dùng Share để chia sẻ liên kết.
- Kaggle: Save notebook để lưu; dùng Publish để chia sẻ phiên bản công khai.
- Azure/AWS: Lưu vào storage liên kết (Azure Blob, S3), sau đó chia sẻ quyền truy cập hoặc xuất notebook.

Mẹo: Đính kèm `requirements.txt` để người khác dễ tái tạo môi trường.

## Lưu ý về TensorFlow 1.x trên Colab
- Google Colab hiện không hỗ trợ `Tensorflow 1` qua `%tensorflow_version 1.x`.
- Có hai hướng:
  1) Chạy với `TensorFlow 2` và bật chế độ tương thích `tf.compat.v1` (đã cấu hình ở ô cài đặt).
  2) Dùng nền tảng khác hỗ trợ TF1 nguyên gốc (Kaggle/Paperspace/AWS với Docker TF1.15).
- Notebook này chọn hướng (1) để chạy nhanh trên Colab.

In [ ]:
# (Tuỳ chọn) Tự động vá import slim từ tf.contrib sang tf_slim
import os, re
def patch_slim_imports(path):
    if not os.path.exists(path):
        return False
    s = open(path,'r',encoding='utf-8',errors='ignore').read()
    changed = False
    # Thay tf.contrib.slim -> tf_slim
    if 'tf.contrib.slim' in s or 'from tensorflow.contrib import slim' in s:
        s = s.replace('tf.contrib.slim', 'tf_slim')
        s = s.replace('from tensorflow.contrib import slim', 'import tf_slim as slim')
        changed = True
    # Nếu code dùng 'import tensorflow as tf; slim = tf.contrib.slim'
    s = re.sub(r"slim\s*=\s*tf\.contrib\.slim", "import tf_slim as slim", s)
    if changed:
        open(path,'w',encoding='utf-8').write(s)
    return changed

# Duyệt một số file mạng phổ biến để vá
targets = [
    'libs/networks/nets_factory.py',
    'libs/networks/resnet_v1.py',
    'libs/networks/resnet_v2.py',
    'libs/networks/mobilenet_v1.py',
    'libs/networks/inception_v3.py',
]
patched = False
for t in targets:
    if patch_slim_imports(t):
        print('Patched:', t)
        patched = True
if not patched:
    print('Không cần vá tf.contrib.slim hoặc không tìm thấy file phù hợp')

In [ ]:
# Gỡ các gói không cần thiết dễ gây xung đột
!pip uninstall -y jax jaxlib tensorflow-decision-forests opencv-python opencv-contrib-python shap pytensor albumentations albucore gradio bigframes rich pillow google-colab ipython jedi requests || true
!pip install -q requests==2.32.4 ipython jedi

In [ ]:
# Đảm bảo jedi đáp ứng yêu cầu của ipython
!pip install -q jedi>=0.16
import jedi, IPython
print('Jedi:', jedi.__version__, 'IPython:', IPython.__version__)